In [2]:
# 06_calculadora_backend.ipynb
# Celda 1
# Carga inicial y unificación de capas

%run ./00_template.py
import geopandas as gpd
import pandas as pd

# 1. Rescatamos el diccionario completo del notebook 02
SERVICE_LAYERS = {
    "salud": "establecimientos_salud",
    "educacion_escolar": "establecimientos_educacion",
    "educacion_superior": "establecimientos_educacion_superior",
    "supermercados": "osm_supermercados",
    "almacenes_barrio": "osm_almacenes_barrio",
    "bancos": "osm_bancos",
    "ferias_libres": "ferias_libres",
    "areas_verdes": "areas_verdes",
    "cuarteles_carabineros": "cuarteles_carabineros",
    "companias_bomberos": "companias_bomberos",
    "estadios": "osm_estadios",
    "malls": "osm_malls",
    "bencineras": "osm_bencineras",
    "iglesias": "osm_iglesias",
    "museos": "osm_museos",
    "infraestructura_deportiva": "infraestructura_deportiva",
    "paradas_micro": "paradas_micro",
    "paradas_metro_tren": "paradas_metro_tren",
}

def cargar_servicios_unificados(gpkg_path, layers_dict):
    """
    Carga todas las capas, les pone una etiqueta y las une en un solo GeoDataFrame.
    """
    lista_gdfs = []
    
    print("Iniciando carga masiva de servicios...")
    for category, layer_name in layers_dict.items():
        try:
            # Cargamos la capa individual
            gdf = gpd.read_file(gpkg_path, layer=layer_name)
            
            # Nos aseguramos de mantener solo la geometría y agregar la categoría
            gdf = gdf[['geometry']].copy()
            gdf['tipo_servicio'] = category
            
            lista_gdfs.append(gdf)
            print(f" {category}: {len(gdf)} puntos")
        except Exception as e:
            print(f" Error cargando {category}: {e}")
    
    # Unimos todo en una sola tabla gigante
    gdf_total = pd.concat(lista_gdfs, ignore_index=True)
    return gdf_total

# EJECUTAR LA CARGA
gdf_servicios = cargar_servicios_unificados(RUTA_GPKG, SERVICE_LAYERS)

# Asegurar sistema de coordenadas métrico (para poder hacer buffers en metros)
# EPSG:32719 es zona UTM 19S (Santiago), ideal para medir metros.
gdf_servicios = gdf_servicios.to_crs(epsg=32719) 

print(f"\n LISTO: Tenemos {len(gdf_servicios)} servicios totales listos para analizar.")

===  Template PEP1 Configurado Exitosamente ===
 Base Dir       : /home/jovyan
 GeoDatabase    :  Encontrada
 Censo CSV      :  Encontrado
Iniciando carga masiva de servicios...
 salud: 1027 puntos
 educacion_escolar: 2918 puntos
 educacion_superior: 526 puntos
 supermercados: 546 puntos
 almacenes_barrio: 2038 puntos
 bancos: 710 puntos
 ferias_libres: 370 puntos
 areas_verdes: 7508 puntos
 cuarteles_carabineros: 128 puntos
 companias_bomberos: 142 puntos
 estadios: 41 puntos
 malls: 144 puntos
 bencineras: 549 puntos
 iglesias: 1028 puntos
 museos: 100 puntos
 infraestructura_deportiva: 4896 puntos
 paradas_micro: 12961 puntos
 paradas_metro_tren: 35 puntos

 LISTO: Tenemos 35667 servicios totales listos para analizar.


In [3]:
# Celda 2
def obtener_servicios_en_radio(lat, lon, radio_metros=1000):
    """
    Dado un punto (lat, lon), cuenta cuántos servicios de cada tipo hay alrededor.
    """
    # 1. Crear un punto con las coordenadas del usuario
    punto_usuario = gpd.GeoDataFrame(
        geometry=gpd.points_from_xy([lon], [lat]), 
        crs="EPSG:4326"
    ).to_crs(gdf_servicios.crs) # Convertir a metros (mismo CRS que la base)
    
    # 2. Crear el buffer (el círculo alrededor del usuario)
    circulo = punto_usuario.buffer(radio_metros)
    
    # 3. Filtrar espacialmente (clip)
    # Esto busca qué servicios caen DENTRO del círculo
    servicios_cercanos = gdf_servicios[gdf_servicios.intersects(circulo.iloc[0])]
    
    # 4. Contar por tipo
    conteo = servicios_cercanos['tipo_servicio'].value_counts().to_dict()
    
    # Rellenar con 0 los servicios que no se encontraron
    for servicio in SERVICE_LAYERS.keys():
        if servicio not in conteo:
            conteo[servicio] = 0
            
    return conteo

In [4]:
# ============================================================================
# CELDA 3: PRUEBA DE FUEGO (NUMÉRICA)
# ============================================================================
# Probemos con una ubicación conocida: Plaza de Armas de Santiago
lat_prueba = -33.4372
lon_prueba = -70.6506

print(f" Analizando ubicación: {lat_prueba}, {lon_prueba}")
print(" Escaneando servicios en 1000 metros a la redonda...")

# Ejecutamos tu función
resultado = obtener_servicios_en_radio(lat_prueba, lon_prueba, radio_metros=1000)

# Mostramos el resultado ordenado
series_resultado = pd.Series(resultado).sort_values(ascending=False)

print("\n RESULTADOS DEL ESCANEO:")
print("-" * 30)
print(series_resultado)
print("-" * 30)
print(f"TOTAL SERVICIOS ENCONTRADOS: {series_resultado.sum()}")

# Cálculo simple de puntaje (Ejemplo: 1 punto por cada tipo de servicio presente)
puntaje_cobertura = (series_resultado > 0).sum()
print(f" PUNTAJE DE DIVERSIDAD: {puntaje_cobertura} / {len(SERVICE_LAYERS)} categorías cubiertas")

 Analizando ubicación: -33.4372, -70.6506
 Escaneando servicios en 1000 metros a la redonda...

 RESULTADOS DEL ESCANEO:
------------------------------
paradas_micro                169
bancos                        90
almacenes_barrio              54
salud                         41
areas_verdes                  35
educacion_superior            28
malls                         22
iglesias                      22
supermercados                 20
museos                        18
infraestructura_deportiva     17
educacion_escolar              7
paradas_metro_tren             5
companias_bomberos             4
cuarteles_carabineros          2
bencineras                     2
ferias_libres                  0
estadios                       0
dtype: int64
------------------------------
TOTAL SERVICIOS ENCONTRADOS: 536
 PUNTAJE DE DIVERSIDAD: 16 / 18 categorías cubiertas


In [5]:
# ============================================================================
# CELDA 4: MOTOR DE NORMALIZACIÓN (SCORING LOGIC)
# ============================================================================

# 1. Configuración Modular (Hash de Reglas)
# 'meta': Cantidad óptima para obtener 100% de puntaje en ese ítem.
# 'desc': Descripción para entender qué estamos midiendo.
SCORING_CONFIG = {
    # Transporte (Saturación rápida: con poco basta)
    "paradas_metro_tren": {"meta": 2, "desc": "Acceso a red principal"},
    "paradas_micro":      {"meta": 5, "desc": "Opciones de recorridos"},
    
    # Salud y Seguridad (Importa la cercanía de al menos uno)
    "salud":                 {"meta": 2, "desc": "Consultorios/Centros Médicos"},
    "cuarteles_carabineros": {"meta": 1, "desc": "Seguridad cercana"},
    "companias_bomberos":    {"meta": 1, "desc": "Respuesta emergencia"},

    # Educación (Variedad es buena)
    "educacion_escolar":  {"meta": 3, "desc": "Opciones colegios"},
    "educacion_superior": {"meta": 1, "desc": "Acceso a educación sup."},

    # Abastecimiento (Saturación media)
    "supermercados":    {"meta": 2, "desc": "Competencia precios"},
    "almacenes_barrio": {"meta": 4, "desc": "Comercio local"},
    "ferias_libres":    {"meta": 1, "desc": "Productos frescos"},
    "bencineras":       {"meta": 2, "desc": "Abastecimiento auto"},

    # Calidad de Vida (Mientras más mejor, hasta cierto punto)
    "areas_verdes":              {"meta": 4, "desc": "Pulmones verdes"},
    "infraestructura_deportiva": {"meta": 3, "desc": "Canchas/Gimnasios"},
    "estadios":                  {"meta": 1, "desc": "Eventos"},
    "malls":                     {"meta": 1, "desc": "Shopping/Ocio"},
    "bancos":                    {"meta": 2, "desc": "Trámites"},
    "iglesias":                  {"meta": 2, "desc": "Culto"},
    "museos":                    {"meta": 1, "desc": "Cultura"}
}

def normalizar_conteo(servicio_key, conteo_real):
    """
    Transforma un número absoluto (ej: 5 farmacias) a un puntaje 0.0 - 1.0
    basado en las reglas de SCORING_CONFIG.
    """
    # Si no tiene regla, asumimos que 1 ya es bueno (default)
    meta = 1 if servicio_key not in SCORING_CONFIG else SCORING_CONFIG[servicio_key]["meta"]
    
    # Fórmula: Porcentaje de cumplimiento de la meta (topeeado de 1.0)
    score = min(conteo_real / meta, 1.0)
    
    return score

# --- PRUEBA RÁPIDA ---
print(" Test de Normalización:")
casos_prueba = [("paradas_metro_tren", 0), ("paradas_metro_tren", 1), ("paradas_metro_tren", 3)]

for serv, n in casos_prueba:
    sc = normalizar_conteo(serv, n)
    print(f"  - Tener {n} {serv}: Puntaje {sc:.2f} (Meta: {SCORING_CONFIG[serv]['meta']})")

 Test de Normalización:
  - Tener 0 paradas_metro_tren: Puntaje 0.00 (Meta: 2)
  - Tener 1 paradas_metro_tren: Puntaje 0.50 (Meta: 2)
  - Tener 3 paradas_metro_tren: Puntaje 1.00 (Meta: 2)


In [6]:
# ============================================================================
# CELDA 5: CONTROLADOR DE PERFILES & CÁLCULO FINAL
# ============================================================================

# 1. Definición de Pesos por Perfil (Escala 1 a 5 de importancia)
# Si un servicio no está en la lista, asumimos importancia 1 (baja)
PERFILES_USUARIO = {
    "estudiante": {
        "pesos": {
            "educacion_superior": 5, "paradas_metro_tren": 5, "paradas_micro": 4,
            "bancos": 3, "malls": 3, "infraestructura_deportiva": 3, 
            "areas_verdes": 2, "bencineras": 1
        },
        "desc": "Prioriza conectividad y educación superior"
    },
    "adulto_mayor": {
        "pesos": {
            "salud": 5, "farmacias": 5, "paradas_micro": 4, # Micro suele ser más accesible que metro
            "areas_verdes": 4, "cuarteles_carabineros": 3,
            "bancos": 3, "ferias_libres": 3,
            "educacion_escolar": 1, "educacion_superior": 1, "infraestructura_deportiva": 1
        },
        "desc": "Prioriza salud, tranquilidad y abasto local"
    },
    "familia_joven": {
        "pesos": {
            "educacion_escolar": 5, "areas_verdes": 5, "salud": 4,
            "supermercados": 4, "cuarteles_carabineros": 4,
            "bencineras": 3, "malls": 3,
            "educacion_superior": 1
        },
        "desc": "Prioriza colegios, parques y seguridad"
    }
}

def calcular_indice_calidad_vida(lat, lon, perfil_key):
    """
    Calcula un puntaje de 0 a 100 para una ubicación dada y un perfil específico.
    Integra: Conteo -> Normalización -> Ponderación.
    """
    # 0. Validar perfil
    if perfil_key not in PERFILES_USUARIO:
        return {"error": f"Perfil '{perfil_key}' no encontrado."}
    
    pesos_perfil = PERFILES_USUARIO[perfil_key]["pesos"]
    
    # 1. Obtener datos crudos (Valentina)
    conteo_servicios = obtener_servicios_en_radio(lat, lon, radio_metros=1000)
    
    puntaje_total_ponderado = 0
    suma_pesos_maximos = 0
    
    detalles = {} # Para entender el puntaje
    
    # 2. Iterar por todos los servicios posibles
    for servicio, conteo in conteo_servicios.items():
        # A. Normalizar -> 0.0 a 1.0
        puntaje_norm = normalizar_conteo(servicio, conteo)
        
        # B. Obtener peso del perfil -> 1 a 5
        peso = pesos_perfil.get(servicio, 1) # Default peso 1 si no es relevante
        
        # C. Acumular
        # El puntaje que aporta este ítem es: (Calidad 0-1) * (Importancia 1-5)
        aporte = puntaje_norm * peso
        puntaje_total_ponderado += aporte
        
        # El máximo posible para este ítem hubiera sido: 1.0 * peso
        suma_pesos_maximos += peso
        
        # Guardar detalle si aportó algo
        if aporte > 0:
            detalles[servicio] = {
                "conteo": conteo,
                "score_norm": round(puntaje_norm, 2),
                "importancia": peso,
                "aporte_final": round(aporte, 2)
            }

    # 3. Calcular Score Final (0 a 100)
    # Es el porcentaje del puntaje obtenido sobre el máximo posible ideal para ese perfil
    if suma_pesos_maximos == 0:
        indice_final = 0
    else:
        indice_final = (puntaje_total_ponderado / suma_pesos_maximos) * 100
        
    return {
        "indice": round(indice_final, 1),
        "perfil": perfil_key,
        "lat": lat, "lon": lon,
        "detalles": detalles
    }

# --- PRUEBA FINAL ---
print(" Test de Cálculo de Índice:")
lat_test, lon_test = -33.4372, -70.6506 # Plaza de Armas

perfiles_a_probar = ["estudiante", "adulto_mayor", "familia_joven"]

for p in perfiles_a_probar:
    res = calcular_indice_calidad_vida(lat_test, lon_test, p)
    print(f"\n Perfil {p.upper()}:")
    print(f"    Índice Calidad de Vida: {res['indice']} / 100")
    # Mostrar top 3 aportes
    top3 = sorted(res['detalles'].items(), key=lambda x: x[1]['aporte_final'], reverse=True)[:3]
    print(f"    Lo que más sumó: {[k for k,v in top3]}")

 Test de Cálculo de Índice:

 Perfil ESTUDIANTE:
    Índice Calidad de Vida: 94.4 / 100
    Lo que más sumó: ['educacion_superior', 'paradas_metro_tren', 'paradas_micro']

 Perfil ADULTO_MAYOR:
    Índice Calidad de Vida: 88.2 / 100
    Lo que más sumó: ['salud', 'paradas_micro', 'areas_verdes']

 Perfil FAMILIA_JOVEN:
    Índice Calidad de Vida: 94.9 / 100
    Lo que más sumó: ['areas_verdes', 'educacion_escolar', 'salud']
